In [1]:
import sys
sys.path.append("code")

import scipy.io as sio
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils import get_ypr_from_mat, get_pt2d_from_mat

/Users/emrecan/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_records = []

datasets = ["AFW", "AFW_Flip", "HELEN", "HELEN_Flip"]
for dataset in datasets:
    mat_files = list(Path(f"data/300W_LP/{dataset}").glob("*.mat"))

    save_cropped_img_path = Path("data/cropped_images")
    save_cropped_img_path.mkdir(exist_ok=True)

    print(f"Processing dataset: {dataset}")
    for mat_file_idx, mat_file in enumerate(tqdm(mat_files)):
        pose = get_ypr_from_mat(mat_file)
        pitch = pose[0] * 180 / np.pi
        yaw = pose[1] * 180 / np.pi
        roll = pose[2] * 180 / np.pi

        pt2d = get_pt2d_from_mat(mat_file)

        x_min = min(pt2d[0,:])
        y_min = min(pt2d[1,:])
        x_max = max(pt2d[0,:])
        y_max = max(pt2d[1,:])

        k = 0.20
        x_min -= 2 * k * abs(x_max - x_min)
        y_min -= 2 * k * abs(y_max - y_min)
        x_max += 2 * k * abs(x_max - x_min)
        y_max += 0.6 * k * abs(y_max - y_min)

        pose_img = cv2.imread(str(mat_file).replace("mat", "jpg"))
        cropped_pose_img = pose_img[int(y_min):int(y_max), int(x_min):int(x_max)]

        img_name = f"{dataset}_{mat_file_idx}.jpg"
        cv2.imwrite(str(save_cropped_img_path.joinpath(img_name)), cropped_pose_img)

        df_records.append(
            {
                "name": img_name,
                "yaw": yaw,
                "pitch": pitch,
                "roll": roll,
            }
        )

        # break

Processing dataset: AFW


100%|██████████| 5207/5207 [00:21<00:00, 237.38it/s]


Processing dataset: AFW_Flip


100%|██████████| 5207/5207 [00:21<00:00, 241.55it/s]


Processing dataset: HELEN


100%|██████████| 37676/37676 [02:59<00:00, 209.69it/s]


Processing dataset: HELEN_Flip


100%|██████████| 37676/37676 [03:26<00:00, 182.09it/s]


In [3]:
df = pd.DataFrame(df_records)

In [4]:
df.head()

,name,yaw,pitch,roll
0,AFW_0.jpg,52.627840,-1.899845,7.454445
1,AFW_1.jpg,-17.416299,-16.421279,-12.825909
2,AFW_2.jpg,55.619767,-29.203772,-17.430045
3,AFW_3.jpg,-15.629833,-1.648596,-9.227935
4,AFW_4.jpg,-17.694836,-9.471268,13.162210


In [5]:
df.to_csv("data/ypr_df.csv", sep=";", index=False)

In [18]:
mat = sio.loadmat("data/300W_LP/landmarks/AFW/AFW_1051618982_1_0_pts.mat")
mat["pts_2d"].shape

(68, 2)

In [17]:
mat = sio.loadmat("data/300W_LP/landmarks/HELEN/HELEN_100032540_1_0_pts.mat")
mat["pts_2d"].shape

(68, 2)